In [ ]:
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import numpy as np
import plotly.express as px
from sympy import symbols
from sympy.physics import mechanics
from numba import jit, njit, numba
from sympy import Dummy, lambdify
from scipy.integrate import odeint
import tensorflow as tf

In [ ]:
def integrate_pendulum(n, times,
                       initial_positions=np.pi/2,
                       initial_velocities=0,
                       lengths=None, masses=1,U_c=0):
    """Integrate a multi-pendulum with `n` sections"""
    #-------------------------------------------------
    # Step 1: construct the pendulum model
    
    # Generalized coordinates and velocities
    # (in this case, angular positions & velocities of each mass) 
    q = mechanics.dynamicsymbols('q:{0}'.format(n))
    u = mechanics.dynamicsymbols('u:{0}'.format(n))

    # mass and length
    m = symbols('m:{0}'.format(n))
    l = symbols('l:{0}'.format(n))

    # gravity and time symbols
    g, t = symbols('g,t')
    
    #--------------------------------------------------
    # Step 2: build the model using Kane's Method

    # Create pivot point reference frame
    A = mechanics.ReferenceFrame('A')
    P = mechanics.Point('P')
    P.set_vel(A, 0)

    # lists to hold particles, forces, and kinetic ODEs
    # for each pendulum in the chain
    particles = []
    forces = []
    kinetic_odes = []

    for i in range(n):
        # Create a reference frame following the i^th mass
        Ai = A.orientnew('A' + str(i), 'Axis', [q[i], A.z])
        Ai.set_ang_vel(A, u[i] * A.z)

        # Create a point in this reference frame
        Pi = P.locatenew('P' + str(i), l[i] * Ai.x)
        Pi.v2pt_theory(P, A, Ai)

        # Create a new particle of mass m[i] at this point
        Pai = mechanics.Particle('Pa' + str(i), Pi, m[i])
        particles.append(Pai)

        # Set forces & compute kinematic ODE
        forces.append((Pi, m[i] * g * A.x))
        forces.append((Pi, U_c[i] * Ai.y))
        kinetic_odes.append(q[i].diff(t) - u[i])

        P = Pi

    # Generate equations of motion
    KM = mechanics.KanesMethod(A, q_ind=q, u_ind=u,
                               kd_eqs=kinetic_odes)
    fr, fr_star = KM.kanes_equations(particles,forces)
    
    #-----------------------------------------------------
    # Step 3: numerically evaluate equations and integrate

    # initial positions and velocities – assumed to be given in degrees
    #y0 = np.deg2rad(np.concatenate([np.broadcast_to(initial_positions, n), np.broadcast_to(initial_velocities, n)]))
    y0 = (np.concatenate([initial_positions, initial_velocities]))
    # lengths and masses
    if lengths is None:
        lengths = np.ones(n) / n
    lengths = np.broadcast_to(lengths, n)
    masses = np.broadcast_to(masses, n)

    # Fixed parameters: gravitational constant, lengths, and masses
    parameters = [g] + list(l) + list(m)
    parameter_vals = [9.81] + list(lengths) + list(masses)

    # define symbols for unknown parameters
    unknowns = [Dummy() for i in q + u]
    unknown_dict = dict(zip(q + u, unknowns))
    kds = KM.kindiffdict()

    # substitute unknown symbols for qdot terms
    mm_sym = KM.mass_matrix_full.subs(kds).subs(unknown_dict)
    fo_sym = KM.forcing_full.subs(kds).subs(unknown_dict)

    # create functions for numerical calculation 
    mm_func = lambdify(unknowns + parameters, mm_sym)
    fo_func = lambdify(unknowns + parameters, fo_sym)

    # function which computes the derivatives of parameters
    def gradient(y, t, args):
        vals = np.concatenate((y, args))
        sol = np.linalg.solve(mm_func(*vals), fo_func(*vals))
        return np.array(sol).T[0]

    # ODE integration
    return odeint(gradient, y0, times, args=(parameter_vals,))

In [ ]:
n=2
dSteps=0.1
T_data = 5000
T_train = 4000
T_test = T_data-T_train
gamma = 1/3
Alpha = 1
Beta = 1
sample_states=(np.random.rand(10,4)*1-1/2).astype('float32')
reference_states=(np.array([0.25,0.25,0,0])).astype('float32')
steps_to_converge=20

In [ ]:
def generate_data():
  state_input=np.zeros((T_data,2*n))
  u_input=np.random.rand(T_data,n)*Beta-Beta/2
  state_output=np.zeros((T_data,2*n))
  t = np.array([dSteps*r for r in range(2)])
  for i in range(T_data):
    x0=np.random.rand(n)*gamma*np.pi - gamma*np.pi/2
    v0=np.random.rand(n)*Alpha*np.pi-Alpha*np.pi/2
    U=u_input[i,:]
    p = integrate_pendulum(n=n, times=t, initial_positions=x0, initial_velocities=v0,U_c=U)
    state_input[i,:]=p[0,:]
    state_output[i,:]=p[1,:]
  return state_input, u_input, state_output


In [ ]:
def pre_processing_data():
  state_input_mapped=np.zeros((T_data,2*n))
  state_output_mapped=np.zeros((T_data,2*n))
  max_angular_velocity = max(np.max(np.abs(state_output[:,n:])),Alpha*np.pi/2)
  max_angular_loc = max(np.max(np.abs(state_output[:,0:n])),gamma*np.pi/2)

  state_input_mapped[:,0:n] = state_input[:,0:n]/max_angular_loc
  state_input_mapped[:,n:] = state_input[:,n:]/max_angular_velocity

  state_output_mapped[:,0:n] = state_output[:,0:n]/max_angular_loc
  state_output_mapped[:,n:] = state_output[:,n:]/max_angular_velocity

  u_input_mapped = u_input/(Beta/2)

  state_input_mapped = (state_input_mapped).astype('float32')
  u_input_mapped = (u_input_mapped).astype('float32')

  data_input_mapped = np.hstack((state_input_mapped,u_input_mapped))
  data_output_mapped = (state_output_mapped).astype('float32')

  input_train_data = data_input_mapped[0:T_train,:]
  output_train_data = data_output_mapped[0:T_train,:]

  input_test_data = data_input_mapped[T_train:,:]
  output_test_data = data_output_mapped[T_train:,:]

  return input_train_data, output_train_data, input_test_data, output_test_data



In [ ]:
state_input, u_input, state_output= generate_data()
input_train_data, output_train_data, input_test_data, output_test_data = pre_processing_data()

In [ ]:
T=20

fig = make_subplots(rows=1, cols=3)
fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    template="simple_white",
)

tr1=go.Scatter(
    x=[r*dSteps for r in range(T)],
    y=input_train_data[0:T,0],
    mode='markers',
    name = 'input_train_sin'
)
tr2=go.Scatter(
    x=[r*dSteps for r in range(T)],
    y=input_train_data[0:T,1],
    mode='markers',
    name = 'input_train_cos'
)
tr3=go.Scatter(
    x=[r*dSteps for r in range(T)],
    y=input_train_data[0:T,-2],
    mode='markers',
    name = 'control_train'
)
fig.add_trace(tr1,col=1,row=1)
fig.add_trace(tr2,col=2,row=1)
fig.add_trace(tr3,col=3,row=1)
fig.show()

In [ ]:
len_states=2*n
class Pend_model(tf.Module):
  def __init__(self,len_states,len_control, **kwargs):
    super().__init__(**kwargs)
    self.number_neurons=20
    self.number_layers=2
    self.wS=tf.Variable(1*tf.random.normal(shape=[len_states+len_control,self.number_neurons],dtype=tf.float32))
    self.bS=tf.Variable(1*tf.random.normal(shape=[1,self.number_neurons],dtype=tf.float32))
    self.wM=tf.Variable(1*tf.random.normal(shape=[self.number_layers,self.number_neurons,self.number_neurons],dtype=tf.float32))
    self.bM=tf.Variable(1*tf.random.normal(shape=[self.number_layers,1,self.number_neurons],dtype=tf.float32))
    self.wE=tf.Variable(1*tf.random.normal(shape=[self.number_neurons,len_states],dtype=tf.float32))
  def __call__(self,x):
    hid = tf.matmul(x,self.wS)+self.bS
    act=tf.nn.tanh(hid)
    for i in range(self.number_layers):
      hid = tf.matmul(act,self.wM[i,:,:])+self.bM[i,:,:]
      act=tf.nn.tanh(hid)
    ybar = tf.matmul(act,self.wE)
    return ybar

class Control_model(tf.Module):
  def __init__(self,len_states,len_input, **kwargs):
    super().__init__(**kwargs)
    self.number_neurons=4
    self.number_layers=1
    self.wS=tf.Variable(1*tf.random.normal(shape=[len_states,self.number_neurons],dtype=tf.float32))
    self.bS=tf.Variable(1*tf.random.normal(shape=[1,self.number_neurons],dtype=tf.float32))
    self.wM=tf.Variable(1*tf.random.normal(shape=[self.number_layers,self.number_neurons,self.number_neurons],dtype=tf.float32))
    self.bM=tf.Variable(1*tf.random.normal(shape=[self.number_layers,1,self.number_neurons],dtype=tf.float32))
    self.wE=tf.Variable(1*tf.random.normal(shape=[self.number_neurons,len_input],dtype=tf.float32))
  def __call__(self,x):
    hid = tf.matmul(x,self.wS)+self.bS
    act=tf.nn.tanh(hid)
    for i in range(self.number_layers):
      hid = tf.matmul(act,self.wM[i,:,:])+self.bM[i,:,:]
      act=tf.nn.tanh(hid)
    ubar = (tf.matmul(act,self.wE))
    return ubar

class Tran_model(tf.Module):
  def __init__(self,len_states, **kwargs):
    super().__init__(**kwargs)
    self.number_neurons=20
    self.number_layers=2
    self.wS=tf.Variable(1*tf.random.normal(shape=[len_states,self.number_neurons],dtype=tf.float32))
    self.bS=tf.Variable(1*tf.random.normal(shape=[1,self.number_neurons],dtype=tf.float32))
    self.wM=tf.Variable(1*tf.random.normal(shape=[self.number_layers,self.number_neurons,self.number_neurons],dtype=tf.float32))
    self.bM=tf.Variable(1*tf.random.normal(shape=[self.number_layers,1,self.number_neurons],dtype=tf.float32))
    self.wE=tf.Variable(1*tf.random.normal(shape=[self.number_neurons,len_states*len_states],dtype=tf.float32))
  def __call__(self,x):
    hid = tf.matmul(x,self.wS)+self.bS
    act=tf.nn.tanh(hid)
    for i in range(self.number_layers):
      hid = tf.matmul(act,self.wM[i,:,:])+self.bM[i,:,:]
      act=tf.nn.tanh(hid)
    mat_flat =tf.nn.tanh(tf.matmul(act,self.wE))
    mat=tf.reshape(mat_flat,(-1,len_states,len_states))
    return mat
  


def loss1(input_data,output_data,pend_model):
    n=input_data.shape[0]
    l1=tf.reduce_mean(tf.square(output_data - pend_model(input_data)))
    return l1



def loss2(input_state,tran_model,pend_model,control_model):
    n=input_state.shape[0]
    dx = np.random.uniform(low=-0.001, high=0.001, size=((10,)+np.shape(input_state)))
    dx = dx.astype('float32') 
    dx_expand=tf.expand_dims(dx, axis=2)
    temp = tf.matmul(dx_expand,tran_model(input_state))
    dz=tf.squeeze(temp)
    dz_norm=tf.reduce_sum(tf.square(dz),axis=2)

    control_signal =  control_model(input_state)
    pert_state=input_state+dx
    control_signal_pert =  control_model(pert_state)

    input = tf.concat([input_state, control_signal], 1)
    input_pert = tf.concat([pert_state, control_signal_pert], 2)

    xp1 = pend_model(input)
    xp1_pert = pend_model(input_pert)
    dxp1 = xp1_pert-xp1

    dxp1_expand=tf.expand_dims(dxp1, axis=2)
    temp = tf.matmul(dxp1_expand,tran_model(xp1))
    dzp1=tf.squeeze(temp)
    dzp1_norm=tf.reduce_sum(tf.square(dzp1),axis=2)
    #l2 = tf.reduce_mean(tf.nn.relu(dzp1_norm-dz_norm))
    l2 = tf.reduce_mean((dzp1_norm-dz_norm))
    return l2    

def loss3(sample_states,reference_states,pend_model,control_model,steps_to_converge):
  temp_state=sample_states
  n=sample_states.shape[0]
  for i in range(steps_to_converge):
    control_signal=control_model(temp_state)
    input_to_dyn_model=tf.concat([temp_state, control_signal], 1)
    temp_state=pend_model(input_to_dyn_model)
  l3=tf.reduce_mean(tf.square(temp_state - reference_states))
  return l3
    
def train_dyn_model(input_data,output_data,pend_model,opt):
    with tf.GradientTape() as t:
      current_loss = loss1(input_data,output_data,pend_model)
    paramsw=[pend_model.wS,pend_model.wM,pend_model.wE]
    paramsb=[pend_model.bS,pend_model.bM]
    grads = t.gradient(current_loss, paramsw+paramsb)
    opt.apply_gradients(zip(grads,paramsw+paramsb))
    

def train_control_tran_model(sample_states,reference_states,steps_to_converge,input_state,tran_model,pend_model,control_model,opt):
    with tf.GradientTape() as t:
      current_loss2 = loss2(input_state,tran_model,pend_model,control_model)
      current_loss3 = loss3(sample_states,reference_states,pend_model,control_model,steps_to_converge)
      current_loss = 100*current_loss2 + current_loss3 
      #current_loss = current_loss2
    paramsw=[control_model.wS,control_model.wM,control_model.wE,tran_model.wS,tran_model.wM,tran_model.wE]
    paramsb=[control_model.bS,control_model.bM,tran_model.bS,tran_model.bM]  
    grads = t.gradient(current_loss, paramsw+paramsb)
    opt.apply_gradients(zip(grads,paramsw+paramsb))



def train_tracking_model(sample_states,reference_states,steps_to_converge,pend_model,control_model,opt):
    with tf.GradientTape() as t:
      current_loss3 = loss3(sample_states,reference_states,pend_model,control_model,steps_to_converge)
      current_loss = current_loss3
    paramsw=[control_model.wS,control_model.wM,control_model.wE]
    paramsb=[control_model.bS,control_model.bM]  
    grads = t.gradient(current_loss, paramsw+paramsb)
    opt.apply_gradients(zip(grads,paramsw+paramsb))


In [ ]:
pend_model=Pend_model(len_states=2*n,len_control=n)

In [ ]:
opt2= tf.keras.optimizers.Adam(learning_rate=0.00001)
for i in range(100001):
  train_dyn_model(input_train_data,output_train_data,pend_model,opt2)
  if i%100==0:
    print('test loss:',loss1(input_test_data,output_test_data,pend_model).numpy())
    print('train loss:',loss1(input_train_data,output_train_data,pend_model).numpy())

test loss: 0.0009718466
train loss: 0.0009658589
test loss: 5.8835634e-05
train loss: 5.7185043e-05
test loss: 4.0044994e-05
train loss: 3.9311682e-05
test loss: 3.9602917e-05
train loss: 3.884544e-05
test loss: 3.9372007e-05
train loss: 3.8604052e-05
test loss: 3.9181436e-05
train loss: 3.8422906e-05
test loss: 3.9011284e-05
train loss: 3.8270948e-05
test loss: 3.8858972e-05
train loss: 3.813868e-05
test loss: 3.872466e-05
train loss: 3.8021175e-05
test loss: 3.8605194e-05
train loss: 3.7914724e-05
test loss: 3.8497274e-05
train loss: 3.7816313e-05
test loss: 3.8398644e-05
train loss: 3.772346e-05
test loss: 3.8307244e-05
train loss: 3.763408e-05
test loss: 3.8219976e-05
train loss: 3.754609e-05
test loss: 3.8135797e-05
train loss: 3.7458703e-05
test loss: 3.805344e-05
train loss: 3.7371345e-05
test loss: 3.7971626e-05
train loss: 3.728331e-05
test loss: 3.789012e-05
train loss: 3.7194513e-05
test loss: 3.7808157e-05
train loss: 3.7104157e-05
test loss: 3.7725018e-05
train loss: 3.701

In [ ]:
x0_valid=np.array([np.pi/6,np.pi/6],dtype='float32')
v0_valid=np.array([-np.pi/12,0],dtype='float32')
U_valid=np.random.rand(T_test,n)*Beta-Beta/2
x0_model_valid=np.concatenate((x0_valid,v0_valid))
T_model_valid=100
t = np.array([dSteps*r for r in range(2)])
max_angular_velocity = max(np.max(np.abs(state_output[:,n:])),Alpha*np.pi/2)
max_angular_loc = max(np.max(np.abs(state_output[:,0:n])),gamma*np.pi/2)


state_valid = np.zeros((T_model_valid,2*n),dtype='float32')
x0=x0_valid
v0=v0_valid
state_valid[0,0:n]=x0/max_angular_loc
state_valid[0,n:]=v0/max_angular_velocity

for i in range(0,T_model_valid-1):
  U=U_valid[i,:]
  p = integrate_pendulum(n=n, times=t, initial_positions=x0, initial_velocities=v0,U_c=U)
  x1=p[1,0:n]
  v1=p[1,n:]
  state_valid[i+1,0:n]=x1/max_angular_loc
  state_valid[i+1,n:]=v1/max_angular_velocity
  x0=x1
  v0=v1


state_model_valid = np.zeros((T_model_valid,2*n),dtype='float32')
x0=x0_valid
v0=v0_valid
state_model_valid[0,0:n]=x0/max_angular_loc
state_model_valid[0,n:]=v0/max_angular_velocity

for i in range(T_model_valid-1):
  U=(U_valid[i,:]/(Beta/2)).astype('float32')
  input_model_valid = np.hstack((x0,v0,U))
  state_model_valid[i+1,:]=pend_model(input_model_valid.reshape((1,6))).numpy()
  x0=state_model_valid[i+1,0:n]
  v0=state_model_valid[i+1,n:]

In [ ]:
T_model_valid=100
t = np.array([0.1*r for r in range(T_model_valid)])
fig=make_subplots(rows=2, cols=2, horizontal_spacing = 0.11)
colors = ['rgb(251,180,174)', 'rgb(0,115,10)', 'rgb(0,0,149)', 'rgb(189,0,189)','rgb(0,210,210)']
colors = ['rgb(251,180,174)', 'rgb(179,205,227)', 'rgb(204,235,197)', 'rgb(222,203,228)','rgb(254,217,166)']
linewidth = [5, 4, 3, 2,1]

fig.update_xaxes(showticklabels=True,dtick = 5,range=[0, 10])
fig.update_xaxes(title_text='Time, t',row=2,col=1,title_standoff=0,dtick = 5,range=[0, 10])
fig.update_xaxes(title_text='Time, t',row=2,col=2,title_standoff=0,dtick = 5,range=[0, 10])
fig.update_yaxes(title_text=r'$\theta^1(\text{Rad})$',row=1,col=1,title_standoff=0)
fig.update_yaxes(title_text=r'$\theta^2(\text{Rad})$',row=1,col=2,title_standoff=0)
fig.update_yaxes(title_text=r'$\dot{\theta}^1(\text{Rad/S})$',row=2,col=1,title_standoff=0)
fig.update_yaxes(title_text=r'$\dot{\theta}^2(\text{Rad/S})$',row=2,col=2,title_standoff=0)



fig.update_layout(
    autosize = False,
    width = 1200,
    height = 600,
    legend = dict(x=1.02,y=0.2,font=dict(size=16)),
    font = dict(size=16,color="#000000",family="Courier New, monospace"),
    template = "simple_white",
)



tr1=go.Scatter(
    x=t,
    y=state_valid[0:T_model_valid,0],
    name="Differential Equation, Angle, Link1"
)

tr2=go.Scatter(
    x=t,
    y=state_model_valid[0:T_model_valid,0],
    name="Neural Network Model, Angle, Link1"
)

tr3=go.Scatter(
    x=t,
    y=state_valid[0:T_model_valid,1],
    name="Differential Equation, Angle, Link2"
)

tr4=go.Scatter(
    x=t,
    y=state_model_valid[0:T_model_valid,1],
    name="Neural Network Model, Angle, Link2"
)

tr5=go.Scatter(
    x=t,
    y=state_valid[0:T_model_valid,2],
    name="Differential Equation, Angular Vel, Link1"
)

tr6=go.Scatter(
    x=t,
    y=state_model_valid[0:T_model_valid,2],
    name="Neural Network Model, Angular Vel, Link1"
)

tr7=go.Scatter(
    x=t,
    y=state_valid[0:T_model_valid,3],
    name="Differential Equation, Angular Vel, Link2"
)

tr8=go.Scatter(
    x=t,
    y=state_model_valid[0:T_model_valid,3],
    name="Neural Network Model, Angular Vel, Link2"
)

fig.add_trace(tr1,row=1,col=1)
fig.add_trace(tr2,row=1,col=1)

fig.add_trace(tr3,row=1,col=2)
fig.add_trace(tr4,row=1,col=2)


fig.add_trace(tr5,row=2,col=1)
fig.add_trace(tr6,row=2,col=1)

fig.add_trace(tr7,row=2,col=2)
fig.add_trace(tr8,row=2,col=2)



fig.show()

In [ ]:
tran_model=Tran_model(len_states=2*n)
control_model=Control_model(len_states=2*n,len_input=n)

In [ ]:
opt1 = tf.keras.optimizers.Adam(learning_rate=0.01)
for i in range(20001):
  train_control_tran_model(sample_states,reference_states,steps_to_converge,output_train_data,tran_model,pend_model,control_model,opt1)
  if i%100==0:
    print('contraction test loss:',loss2(output_test_data,tran_model,pend_model,control_model).numpy())
    print('contraction train loss:',loss2(output_train_data,tran_model,pend_model,control_model).numpy())
    print('convergance loss:',loss3(sample_states,reference_states,pend_model,control_model,steps_to_converge).numpy())
    print(i)

In [ ]:
x0_model_controlled=np.array([0,0,0,0])
T_model_controlled=100
u_rec=np.zeros((T_model_controlled,n),dtype='float32')
x_controlled_model=np.zeros((T_model_controlled,2*n),dtype='float32')
x_controlled_model[0,:]=x0_model_controlled
for i in range(T_model_controlled-1):
    u_control=control_model(x_controlled_model[i,:].reshape((1,2*n)))
    u_rec[i,:]=u_control
    data= np.concatenate((x_controlled_model[i,:],u_control[0,:])).astype('float32')
    x_controlled_model[i+1,:]=pend_model(data.reshape((1,2*n+n))).numpy()

In [ ]:
fig=make_subplots(rows=2,cols=2, horizontal_spacing = 0.12)
colors = ['rgb(251,180,174)', 'rgb(0,115,10)', 'rgb(0,0,149)', 'rgb(189,0,189)','rgb(0,210,210)']
colors = ['rgb(251,180,174)', 'rgb(179,205,227)', 'rgb(204,235,197)', 'rgb(222,203,228)','rgb(254,217,166)']
linewidth = [5, 4, 3, 2,1]

fig.update_xaxes(showticklabels=True,dtick = 20,range=[0, 80])
fig.update_xaxes(title_text='Time, t',row=2,col=1,title_standoff=0,dtick = 20,range=[0, 80])
fig.update_xaxes(title_text='Time, t',row=2,col=2,title_standoff=0,dtick = 20,range=[0, 80])
fig.update_yaxes(title_text=r'$\theta^1(\text{Rad}),\dot{\theta}^1(\text{Rad/S})$',row=1,col=1,title_standoff=0)
fig.update_yaxes(title_text=r'$U_1$',row=1,col=2,title_standoff=0)
fig.update_yaxes(title_text=r'${\theta}^2(\text{Rad}),\dot{\theta}^2(\text{Rad/S})$',row=2,col=1,title_standoff=0)
fig.update_yaxes(title_text=r'$U_2$',row=2,col=2,title_standoff=0)


fig.update_layout(
    autosize = False,
    width = 1200,
    height = 600,
    legend = dict(x=1.05,y=0.3,font=dict(size=16)),
    font = dict(size=16,color="#000000",family="Courier New, monospace"),
    template = "simple_white",
)

t1=go.Scatter(
    x=[r for r in range(T_model_controlled)],
    y=x_controlled_model[0:T_model_controlled,0]*3.27,
    name = "Controled Model Angle Link1"
)
t2=go.Scatter(
    x=[r for r in range(T_model_controlled)],
    y=x_controlled_model[0:T_model_controlled,2],
    name = "Controled Model Angular Vel Link1"
)
t3=go.Scatter(
    x=[r for r in range(T_model_controlled)],
    y=u_rec[0:T_model_controlled-1,0],
    name = "Control input Link1"
)

t4=go.Scatter(
    x=[r for r in range(T_model_controlled)],
    y=x_controlled_model[0:T_model_controlled,1]*3.27,
    name = "Controled Model Angle Link2"
)
t5=go.Scatter(
    x=[r for r in range(T_model_controlled)],
    y=x_controlled_model[0:T_model_controlled,3],
    name = "Controled Model Angular Vel Link2"
)
t6=go.Scatter(
    x=[r for r in range(T_model_controlled)],
    y=u_rec[0:T_model_controlled-1,1],
    name = "Control input Link2"
)



fig.add_trace(t1,row=1,col=1)
fig.add_trace(t2,row=1,col=1)
fig.add_trace(t3,row=1,col=2)

fig.add_trace(t4,row=2,col=1)
fig.add_trace(t5,row=2,col=1)
fig.add_trace(t6,row=2,col=2)

fig.show()

In [ ]:
x0_sys_controlled=np.array([0,0,0,0])
T_sys_controlled=100
t = np.array([dSteps*r for r in range(2)])
u_rec=np.zeros((T_sys_controlled,n))
x_controlled_sys=np.zeros((T_sys_controlled,2*n))
x_controlled_sys[0,:]=x0_sys_controlled

x0=x0_sys_controlled[0:n]*max_angular_loc
v0=x0_sys_controlled[n:]*max_angular_velocity


for i in range(0,T_sys_controlled-1):
  U=control_model(x_controlled_sys[i,:].reshape((1,2*n)).astype('float32')).numpy().astype('float64')
  u_rec[i,:]=U
  p = integrate_pendulum(n=n, times=t, initial_positions=x0, initial_velocities=v0,U_c=Beta*U[0,:])
  x1=p[1,0:n]
  v1=p[1,n:]
  x_controlled_sys[i+1,0:n]=x1/max_angular_loc
  x_controlled_sys[i+1,n:]=v1/max_angular_velocity
  x0=x1
  v0=v1


In [ ]:
fig=make_subplots(rows=2,cols=2, horizontal_spacing = 0.12)
colors = ['rgb(251,180,174)', 'rgb(0,115,10)', 'rgb(0,0,149)', 'rgb(189,0,189)','rgb(0,210,210)']
colors = ['rgb(251,180,174)', 'rgb(179,205,227)', 'rgb(204,235,197)', 'rgb(222,203,228)','rgb(254,217,166)']
linewidth = [5, 4, 3, 2,1]

fig.update_xaxes(showticklabels=True,dtick = 20,range=[0, 80])
fig.update_xaxes(title_text='Time, t',row=2,col=1,title_standoff=0,dtick = 20,range=[0, 80])
fig.update_xaxes(title_text='Time, t',row=2,col=2,title_standoff=0,dtick = 20,range=[0, 80])
fig.update_yaxes(title_text=r'$\theta^1(\text{Rad}),\dot{\theta}^1(\text{Rad/S})$',row=1,col=1,title_standoff=0,dtick = 0.2)
fig.update_yaxes(title_text=r'$U_1$',row=1,col=2,title_standoff=0)
fig.update_yaxes(title_text=r'${\theta}^2(\text{Rad}),\dot{\theta}^2(\text{Rad/S})$',row=2,col=1,title_standoff=0)
fig.update_yaxes(title_text=r'$U_2$',row=2,col=2,title_standoff=0)


fig.update_layout(
    autosize = False,
    width = 1200,
    height = 600,
    legend = dict(x=1.05,y=0.3,font=dict(size=16)),
    font = dict(size=16,color="#000000",family="Courier New, monospace"),
    template = "simple_white",
)

t1=go.Scatter(
    x=[r for r in range(T_sys_controlled)],
    y=x_controlled_sys[0:T_sys_controlled,0]*1.7,
    name = "Controled Dynamics Angle Link1"
)
t2=go.Scatter(
    x=[r for r in range(T_sys_controlled)],
    y=x_controlled_sys[0:T_sys_controlled,2],
    name = "Controled Model Angular Vel Link1"
)
t3=go.Scatter(
    x=[r for r in range(T_sys_controlled)],
    y=u_rec[0:T_sys_controlled-1,0],
    name = "Control input Link1"
)

t4=go.Scatter(
    x=[r for r in range(T_sys_controlled)],
    y=x_controlled_sys[0:T_sys_controlled,1]*1.7,
    name = "Controled Dynamics Angle Link2"
)
t5=go.Scatter(
    x=[r for r in range(T_sys_controlled)],
    y=x_controlled_sys[0:T_sys_controlled,3],
    name = "Controled Model Angular Vel Link2"
)
t6=go.Scatter(
    x=[r for r in range(T_sys_controlled)],
    y=u_rec[0:T_sys_controlled-1,1],
    name = "Control input Link2"
)


fig.add_trace(t1,row=1,col=1)
fig.add_trace(t2,row=1,col=1)
fig.add_trace(t3,row=1,col=2)

fig.add_trace(t4,row=2,col=1)
fig.add_trace(t5,row=2,col=1)
fig.add_trace(t6,row=2,col=2)

fig.show()

In [ ]:
while True:
  pass

KeyboardInterrupt: ignored